# TP1
## Grupo 17:
**PG50315 - David Alexandre Ferreira Duarte**

**PG51247 - João Rafael Cerqueira Monteiro**

## Exercício 1. 
Use a package Criptography  para 
Criar um comunicação privada assíncrona entre um agente Emitter e um agente Receiver que cubra os seguintes aspectos:
    1. Autenticação do criptograma e dos metadados (associated data). Usar uma cifra simétrica  num modo **HMAC**  que seja seguro contra ataques aos “nounces” .
    2. Os “nounces” são gerados por um gerador pseudo aleatório (PRG) construído por um função de hash em modo XOF.
    3. O par de chaves $$cipher\_key$$, $$mac\_key$$ , para cifra e autenticação, é acordado entre agentes usando o protocolo ECDH com autenticação dos agentes usando assinaturas ECDSA.

In [1]:
!pip install cryptography
!pip install nest_asyncio

In [2]:
# Imports
import os
import asyncio
from pickle import dumps, loads
from cryptography.hazmat.primitives import hashes, hmac, serialization
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.serialization import load_pem_public_key
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.exceptions import InvalidSignature


In [3]:
async def gerarChaves():
    cifraChavePrivada = ec.generate_private_key()
    cifraChavePublica = cifraChavePrivada.public_key()
    
    macChavePrivada = ec.generate_private_key()
    macChavePublica = macChavePrivada.public_key()

    mensagem = {'sms_cipher': cifraChavePublica.public_bytes(encoding=serialization.Encoding.PEM, format=serialization.PublicFormat.SubjectPublicKeyInfo), 
                'sms_mac': macChavePublica.public_bytes(encoding=serialization.Encoding.PEM, format= serialization.PublicFormat.SubjectPublicKeyInfo)}

    return dumps(mensagem), cifraChavePrivada, macChavePrivada


In [4]:
async def gerarAssinatura(sms, chavePrivada):
    
    assinatura = chavePrivada.sign(sms, hashes.SHA256())
    
    smsFinal = {'message': sms, 
                'signature': assinatura, 
                'publicKey': chavePrivada.public_key().public_bytes(encoding=serialization.Encoding.PEM, format=serialization.PublicFormat.SubjectPublicKeyInfo)}
    
    return smsFinal

In [5]:
async def gerarChavePartilhada(sms, cifraChavePrivada, macChavePrivada):
    cifraChavePublica = load_pem_public_key(sms['sms_cipher'])
    macChavePublica = load_pem_public_key(sms['sms_mac'])

    cifraChavePartilhada = cifraChavePrivada.exchange(cifraChavePublica)
    macChavePartilhada = macChavePrivada.exchange(macChavePublica)

    return cifraChavePartilhada, macChavePartilhada


In [6]:
async def gerarMac(chave, crypto):
    h = hmac.HMAC(chave, hashes.SHA256(), backend=default_backend())
    h.update(crypto)
    return h.finalize()

In [7]:
async def cifrar(plaintext, cifraChave, macChave, additionalData):
    xof = hashes.Hash(hashes.SHAKE256(), backend=default_backend())
    xof.update(b'nounce generator')
    nounce = xof.finalize(12)

    encryptor = Cipher(algorithms.AES(cifraChave), modes.GCM(nounce), backend=default_backend()).encryptor()

    encryptor.authenticate_additional_data(additionalData)

    ciphertext = encryptor.update(plaintext.encode()) + encryptor.finalize()

    sms = {'nounce': nounce,
           'tag': encryptor.tag,
           'cipher': ciphertext}

    hmac_key = gerarMac(macChave, dumps(sms))

    return {'message': sms, 'tag': hmac_key, 'associated_data': additionalData}


In [8]:
async def decifrar(ciphertext, cifraChave, macChave):
    texto = ciphertext['message']
    hmac_key = ciphertext['tag']

    macDestino = gerarMac(macChave, dumps(texto))

    if hmac_key != macDestino:
        return 'ERRO - MAC não é igual'

    xof = hashes.Hash(hashes.SHAKE256(), backend=default_backend())
    xof.update(b'nonce generator')
    nounce = xof.finalize(12)

    decryptor = Cipher(algorithms.AES(cifraChave), modes.GCM(nounce, texto['tag']), backend=default_backend()).decryptor()

    decryptor.authenticate_additional_data(texto['associated_data'])

    plaintext = decryptor.update(texto['cipher']) + decryptor.finalize()

    return plaintext.decode()


In [9]:
async def Emitter():
    sms, cifraChavePrivada, macChavePrivada = gerarChaves()
    smsFinal = gerarAssinatura(sms)
    
    #enviar smsFinal 
    
    # sms = receber sms do outro lado
    
    ecdsaPublico = load_pem_public_key(sms['publicKey'])
    
    try:
        ecdsaPublico.verify(sms['signature'], sms['message'], hashes.SHA256())
        
        pacoteSMS = loads(sms['message'])
        cifraChavePartilhada, macChavePartilhada = gerarChavePartilhada(pacoteSMS, cifraChavePrivada, macChavePrivada)
        
        texto = "Mensagem do emitter para o receiver"
        
        dadosAssociados = os.urandom(16)
        
        hmacChave = gerarMac(macChavePartilhada, macChavePartilhada)
        
        mensagem = cifrar(texto, cifraChavePartilhada, macChavePartilhada, dadosAssociados)
        
        mensagem['hmac_key'] = hmacChave
        mensagem['associated_data'] = dadosAssociados
        
        # enviar mensagem
        
    except InvalidSignature:
        print("ERRO: Mensagem não autenticada")

In [18]:
async def Receiver():
    
    pkg, cifraChavePrivada, macChavePrivada = gerarChaves()
    finalPkg = generateSignature(pkg)

    # sms = receber do outro lado

    ecdsaPublico = load_pem_public_key(msg['pub_key'])
    
    try:
        ecdsaPublico.verify(msg['signature'],msg['message'],hashes.SHA256())
        #geração das chaves partilhadas
        pkg_msg = loads(msg['message'])
        cifraChavePartilhada, macChavePartilhada = gerarChavePartilhada(pkg_msg,cifraChavePrivada,macChavePrivada)
        # enviar finalPkg 

        # message = receber do outro lado
        print(message)
        hmac_key = message['hmac_key']
        associatedData = message['associated_data']
        #verificar o código de autenticação
        if hmac_key == gerarMac(sharedKey_mac,sharedKey_mac):
        #decifrar a mensagem
            final_message = decifrar(message, sharedKey_cipher, sharedKey_mac,associatedData)
        else:
            print('ERRO - Chaves diferentes em usi.')
    except InvalidSignature:
        print("ERRO: A mensagem não é autenticada.")
